In [1]:
import os, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import time
import shutil
import collections
from pathlib import Path

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Loading packages

In [3]:
import sys
from pathlib import Path

here_path = Path().resolve()
repo_path = here_path.parents[1]
sys.path.append(str(repo_path))

In [4]:
from py.utils import verifyDir, verifyFile, verifyType

In [5]:
from py.config import Config

cfg = Config()

np.random.seed(cfg.RANDOM_STATE)
cfg.DATA_PATH, cfg.MODEL_PATH

('/media/felipe/DATA19/datasets/', '/media/felipe/DATA19/models/')

In [6]:
QSCORE_PATH=f"{cfg.DATA_PATH}pp2/{cfg.SCORING_METHOD}/{cfg.PLACE_LEVEL}/"
IMAGES_PATH = f"{cfg.DATA_PATH}pp2/images/"

CITYSCAPES_DIR = f"{cfg.DATA_PATH}CityScapes/"
ADE20K_DIR = f"{cfg.DATA_PATH}ADE20k/"

In [7]:
verifyDir(CITYSCAPES_DIR)

#### Loading segmentations

In [8]:
from py.models.segmentation.cnn import ConvMaskClassifier

model = ConvMaskClassifier(dataset="cityscapes", load_model=False)

In [9]:
cityscape_data_dict = model._get_color_dict()
cityscape_data_dict

[(0, 'background', (0, 0, 0)),
 (1, 'road', (244, 35, 232)),
 (2, 'sidewalk', (70, 70, 70)),
 (3, 'building', (102, 102, 156)),
 (4, 'wall', (190, 153, 153)),
 (5, 'fence', (153, 153, 153)),
 (6, 'pole', (250, 170, 30)),
 (7, 'traffic_light', (220, 220, 0)),
 (8, 'traffic_sign', (107, 142, 35)),
 (9, 'vegetation', (152, 251, 152)),
 (10, 'terrain', (0, 130, 180)),
 (11, 'sky', (220, 20, 60)),
 (12, 'person', (255, 0, 0)),
 (13, 'rider', (0, 0, 142)),
 (14, 'car', (0, 0, 70)),
 (15, 'truck', (0, 60, 100)),
 (16, 'bus', (0, 80, 100)),
 (17, 'train', (0, 0, 230)),
 (18, 'motorcycle', (119, 11, 32)),
 (19, 'bicycle', (128, 64, 128))]

In [10]:
from py.utils.color import rgb2hex

cityscapes_df = pd.DataFrame(cityscape_data_dict, columns=['class_id', 'class_name', "RGB_color"])
cityscapes_df["main_class"] = cityscapes_df["class_name"].apply(lambda x: "_".join(x.split()))
cityscapes_df["hex_color"] = cityscapes_df["RGB_color"].apply(rgb2hex)
cityscapes_df

,class_id,class_name,RGB_color,main_class,hex_color
0,0,background,"(0, 0, 0)",background,#000000
1,1,road,"(244, 35, 232)",road,#f423e8
2,2,sidewalk,"(70, 70, 70)",sidewalk,#464646
3,3,building,"(102, 102, 156)",building,#66669c
4,4,wall,"(190, 153, 153)",wall,#be9999
5,5,fence,"(153, 153, 153)",fence,#999999
6,6,pole,"(250, 170, 30)",pole,#faaa1e
7,7,traffic_light,"(220, 220, 0)",traffic_light,#dcdc00
8,8,traffic_sign,"(107, 142, 35)",traffic_sign,#6b8e23
9,9,vegetation,"(152, 251, 152)",vegetation,#98fb98


In [11]:
isthing_df = pd.read_csv(f"{ADE20K_DIR}ade20k_categories.csv", sep=";", low_memory=False)
isthing_df.drop(columns=["RGB_color", "hex_color"], inplace=True)
isthing_df

,class_id,main_class,class_name,isthing,group_name
0,1,wall,wall,0,construction
1,2,building,building;edifice,0,construction
2,3,sky,sky,0,sky
3,4,floor,floor;flooring,0,floor
4,5,tree,tree,0,vegetation
...,...,...,...,...,...
145,146,shower,shower,0,indoor_object
146,147,radiator,radiator,1,terrain_vehicle_related
147,148,glass,glass;drinking;glass,1,miscellaneous
148,149,clock,clock,1,miscellaneous


In [12]:
cde_df = pd.merge(cityscapes_df, isthing_df[["main_class", "isthing", "group_name"]].copy(), how="left", on="main_class")
# Completing group names
cde_df.loc[(cde_df["main_class"]=="background"), "isthing"] = -1
cde_df.loc[(cde_df["main_class"]=="background"), "group_name"] = "background"

cde_df.loc[(cde_df["main_class"]=="traffic_light"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="traffic_light"), "group_name"] = "city_elements"

cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="traffic_sign"), "group_name"] = "city_elements"

cde_df.loc[(cde_df["main_class"]=="vegetation"), "isthing"] = 0
cde_df.loc[(cde_df["main_class"]=="vegetation"), "group_name"] = "vegetation"

cde_df.loc[(cde_df["main_class"]=="terrain"), "isthing"] = 0
cde_df.loc[(cde_df["main_class"]=="terrain"), "group_name"] = "floor"

cde_df.loc[(cde_df["main_class"]=="rider"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="rider"), "group_name"] = "human"

cde_df.loc[(cde_df["main_class"]=="train"), "isthing"] = 1
cde_df.loc[(cde_df["main_class"]=="train"), "group_name"] = "terrain_vehicle"

# Groups
cde_df.loc[(cde_df["group_name"]=="background"), "RGB_color_group"] = "(0, 0, 0)"
cde_df.loc[(cde_df["group_name"]=="terrain_vehicle"), "RGB_color_group"] = "(255, 0, 20)"
cde_df.loc[(cde_df["group_name"]=="floor"), "RGB_color_group"] = "(140, 140, 140)"
cde_df.loc[(cde_df["group_name"]=="construction"), "RGB_color_group"] = "(180, 120, 120)"
cde_df.loc[(cde_df["group_name"]=="city_elements"), "RGB_color_group"] = "(0, 71, 255)"
cde_df.loc[(cde_df["group_name"]=="vegetation"), "RGB_color_group"] = "(4, 200, 3)"
cde_df.loc[(cde_df["group_name"]=="sky"), "RGB_color_group"] = "(6, 230, 230)"
cde_df.loc[(cde_df["group_name"]=="human"), "RGB_color_group"] = "(150, 5, 61)"

cde_df["hex_color_group"] = cde_df["RGB_color_group"].apply(rgb2hex)
cde_df.sort_values(by="group_name", inplace=True)
cde_df.reset_index(inplace=True, drop=True)
cde_df

,class_id,class_name,RGB_color,main_class,hex_color,isthing,group_name,RGB_color_group,hex_color_group
0,0,background,"(0, 0, 0)",background,#000000,-1.0,background,"(0, 0, 0)",#000000
1,6,pole,"(250, 170, 30)",pole,#faaa1e,1.0,city_elements,"(0, 71, 255)",#0047ff
2,7,traffic_light,"(220, 220, 0)",traffic_light,#dcdc00,1.0,city_elements,"(0, 71, 255)",#0047ff
3,8,traffic_sign,"(107, 142, 35)",traffic_sign,#6b8e23,1.0,city_elements,"(0, 71, 255)",#0047ff
4,3,building,"(102, 102, 156)",building,#66669c,0.0,construction,"(180, 120, 120)",#b47878
5,4,wall,"(190, 153, 153)",wall,#be9999,0.0,construction,"(180, 120, 120)",#b47878
6,5,fence,"(153, 153, 153)",fence,#999999,1.0,construction,"(180, 120, 120)",#b47878
7,1,road,"(244, 35, 232)",road,#f423e8,0.0,floor,"(140, 140, 140)",#8c8c8c
8,2,sidewalk,"(70, 70, 70)",sidewalk,#464646,0.0,floor,"(140, 140, 140)",#8c8c8c
9,10,terrain,"(0, 130, 180)",terrain,#0082b4,0.0,floor,"(140, 140, 140)",#8c8c8c


In [13]:
cde_df.to_csv(f"{CITYSCAPES_DIR}cityscapes_categories.csv", sep=";", index=False)